# Engineering Notebook: Design Exercise 1

### Ideas for the Wire Protocol

- Use TCP/IP as template
    - Do we need IP addresses in the header?
    - How can we verify packets?
- VBS: Should we use some kind of compression for the message contents?
    - VBS: Maybe not, let's keep it simple
- Features:
    - Header version
    - Header checksum
    - Content checksum

- What type of data are we sending across?
    - Operations + arguments (e.g. page/pagination, optional arguments)
    - Strings (e.g. name, password hash, error messages)
    - Status code (from the server to detect errors)
    - Arrays of strings -> could be sent as string of strings separated by commas like a CSV

- How do you specify which messages you want to delete?
    - Could go by message ID individually
    - Could go by sender/receiver name